# 数模线代

### 3.1 特征值与特征向量

3.1.1 差分方程
差分方程形如以下形式：
$$
    a_n = \alpha a_{n-1} + \beta a_{n-2} + \dots
$$
一般遇到的都是二阶的

对于其的求解，可以采用Jordon对角化来计算
对于特征方程
$$
    \lambda^2 + \alpha \lambda + \beta = 0 
$$
若判别式$\Delta$等于零时，不能相似对角化，故采用更一般的Jordon对角化

In [ ]:
import sympy as sp
k = sp.var('k', integer=True, positive=True)
# 正常有两个特征值的情况，形如 a_n = c1 * lambda1^n + c2 * lambda2^n 
a = sp.Matrix([[1, 1], [0, 1]]) 
# 有一个特征值的情况,形如 a_n = （c1 + c2 * n）* lambda^n
# a= sp.Matrix([[4, -4], [0, 1]])
# 有复数特征值的情况，结果形如 a_n = c1 * lambda1^n + c2 * lambda2^n
# 结果有时会出现一定的周期性，此例为周期为6c
# a = sp.Matrix([[1, -1], [1, 0]])
P , J = a.jordan_form()
ak = P @ ( J** k ) @ (P.inv())
F = ak @ sp.Matrix([1, 0])
s = sp.simplify(F[0])
print(s)
sm = []
for i in range(10):
    sm.append((s.subs(k, i)).simplify())
print(sm)

3.1.2 Leslie种族模型

其将种族给分为n个年龄组，各个年龄组的生育率与存活率分别为$a_i,b_i$

将种族各年龄组数量$x_i$列为向量：
$X^{(t)} =[x_1^{(t)},x_2^{(t)},x_3^{(t)},...,x_n^{(t)}] $

则$X^{(n+1)} = L X^{(n)}$,其中L为：

L = $
    \begin{bmatrix}
        a_1 & a_2 & \cdots & a_{n-1} & a_n \\
        b_1 &  0  & \cdots &   0    &  0  \\
        0   & b_2 & \cdots &   0    &  0  \\
        \vdots & \vdots & \ddots & \vdots & \vdots \\
        0   &  0  & \cdots & b_{n-1} &  0
    \end{bmatrix}
$

不妨设L具有n个对应的特征值与n个线性无关的特征向量，那么X可以由其表示，设为：
$$
    X^{(0)} =c_1 X_1 +c_2  X_2 + \cdots + c_n X_n
$$
那么
$$
    X^{(n)} =c_1 \lambda_1^n X_1 + c_2 \lambda_2^n X_2 + \cdots +c_3 \lambda_n^3 X_n
$$
不难得知，当迭代的次数趋于无穷时,$X^{(n)}$,可近似表示为：$X^{(n)}=c_m \lambda_m^n X_m $

其中$\lambda_m=max{|\lambda_i|}$

故由此可知，种族最后稳定的比例近似为特征值最大的对应特征向量的比例

3.1.3 PageRank算法

构造一个有向图$D = (V,\widetilde{A},W)$,其中$V={v_1,v_2,...,v_n}$为顶点的合集

而$\widetilde{A}$为弧的集合，W为其邻接矩阵

如果我们认为用户在此页面时，会等可能的转移到其他的页面时

不难得到，对应的状态转移概率矩阵P，记W的行和$r_i = \sum_{j=0}^{n-1} a_{ij}$,则
$$
    p_{ij} = w_{ij}/r_i
$$
设用户在第n个时刻在第各页面的比例为：$X^n = [x_1,x_2,...x_i]$,其中$\sum x_i =1$

那么可知：$X^n = (P^T)^n X^0$

若$X^n$有极限的话，这将会满足$X = P^T X$,即X为$P^T$对应特征值为1的特征向量

不难得知，P的每一列和均为1，必有一个特征值为1，故X也必定存在，关于X的求解即是求解对应的特征向量（注意一下，要较好的求解，建议求解（P - E）=0 这个方程，直接用py求解特征值时带上了一个极小i，对应的特征向量也带上虚数单位）

plus版的算法认为用户只有d的概率沿着url前进，有（1-d）的概率跳到其页面，其中认为跳转到其他页面是等可能的，那么不难将概率转移矩阵修正为：$\widetilde{P} = \frac{1-d}{n} + d*P$,其他的同理。

合理猜想（无证明），极限X为P（或$\widetilde{P}$）的一个稳定的不动点，参考数列的情况，但并不能知道是否是一个局域稳定不动点还是全局稳定不动点，这是根据$X^n$和为1，又非负，则$X^n$的取值集合是一个有限的高维平面，而且在这个区域内有一个不动点，故猜想，下面代码即是验证：
取初始$X^0$满足初始条件任意，都收敛到例题给的结果

In [ ]:
import numpy as np
import sympy as sp
A = sp.Matrix([[0,1,0,0,0,0],
               [0,0,1/2,1/2,0,0],
               [0,0,0,1/3,1/3,1/3],
               [1,0,0,0,0,0],
               [0,0,0,0,0,1],
               [1,0,0,0,0,0]]).T
a_0 = sp.Matrix([1/7, 2/7, 1/7, 1/7, 1/7, 1/7])
for i in range(200) :
    a_0 = A @ a_0
print(a_0)
B = np.array([[0,1,0,0,0,0],
                [0,0,1/2,1/2,0,0],
                [0,0,0,1/3,1/3,1/3],
                [1,0,0,0,0,0],
                [0,0,0,0,0,1],
                [1,0,0,0,0,0]]).T
print(np.linalg.det(B-1))
B = (1-0.85)/6 + 0.85 * B
b_0 = np.array([1/6, 1/6, 1/6, 1/6, 1/6, 1/6])
for i in range(100) :
    b_0 = B @ b_0
print(b_0)